In [1]:
#librerias
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import seaborn as sns
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import plot_confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [2]:
#instalando librerias de nltk
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
#queremos resumir el texto usamos nltk. descargamos librería nltk para usar stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/carolinaruiz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
#stopwords en inglés (articulos, pronombres,preposiciones...) con este stopwords lo que conseguimos es que si en el texto aparecen alguna de estas palabras se eliminen para asi hacer más resumido el texto
print(stopwords.words('English'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
#Importando data
df =pd.read_csv('/Users/Carolinaruiz/Desktop/THEBRIDGE/proyecto/train.csv')

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
#id= id unico para cada artículo
#title= título del artículo
#author=autor de cada artículo
#text= parte del texto del artículo
# label: 0 noticia real 1 fake 

In [10]:
#vemos a ver si tiene nulos
df.info(memory_usage='deep') 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 186.5 MB


In [11]:
#vemos cuantos nulos
df.isnull().sum() 

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
#sustituimos los valores nulos por un ''
df=df.fillna('')

In [14]:
#vemos que la columna id no nos sirve de nada y la eliminamos
df=df.drop('id',axis=1)

In [15]:
df.info(memory_usage='deep')#comprobamos que hemos sustiuido los nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   20800 non-null  object
 1   author  20800 non-null  object
 2   text    20800 non-null  object
 3   label   20800 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 186.4 MB


In [16]:

df['info']=df['author']+'  '+df['title']

In [17]:
print(df['info'])

0        Darrell Lucus  House Dem Aide: We Didn’t Even ...
1        Daniel J. Flynn  FLYNN: Hillary Clinton, Big W...
2        Consortiumnews.com  Why the Truth Might Get Yo...
3        Jessica Purkiss  15 Civilians Killed In Single...
4        Howard Portnoy  Iranian woman jailed for ficti...
                               ...                        
20795    Jerome Hudson  Rapper T.I.: Trump a ’Poster Ch...
20796    Benjamin Hoffman  N.F.L. Playoffs: Schedule, M...
20797    Michael J. de la Merced and Rachel Abrams  Mac...
20798    Alex Ansary  NATO, Russia To Hold Parallel Exe...
20799             David Swanson  What Keeps the F-35 Alive
Name: info, Length: 20800, dtype: object


In [18]:
#separamos el label del resto
X=df.drop(columns='label',axis=1)
Y= df['label']


In [19]:
#para ver cuanto tenemos de false y cuanto de true 
#vamos que esta bastante igualado 
df_train['label'].value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [20]:
#transformación
port_stem=PorterStemmer()


#df ['info']=df['info'].str.lower()
#df['info']= df['info'].str.split()
#df['info']= ' '.join (df)
#metemos en una función toda la transformación
def data_transformation(info):

    letters_only = re.sub("[^a-zA-Z]",  
                          " ",         
                          str (info))     
    words = letters_only.lower().split() 
    stops = set(stopwords.words("english")) 
    meaningful_words = [port_stem.stem(w) for w in words if not w in stops] 
   
    return (" ".join(meaningful_words)) 

In [21]:
df['info']=df['info'].apply(data_transformation) #aplicamos la transformación a la columna info que es la que nos interesa

In [23]:
#vemos como ha quedado
print(df['info'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: info, Length: 20800, dtype: object


In [24]:
#separamos info de label
X=df['info'].values
Y=df['label'].values

In [25]:
#vamos a tranformar el texto a numérico 
vectorizer= TfidfVectorizer()
vectorizer.fit(X)

X= vectorizer.transform(X)

In [26]:
print (X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [27]:
#ENTRENAMIENTO

In [29]:

X_train, X_test, Y_train, Y_test = tts(X, Y, test_size=0.2, random_state=0)


In [30]:

model=LogisticRegression()

In [31]:

model.fit(X_train, Y_train)

LogisticRegression()

In [33]:

predictions_train=model.predict(X_train)
train_score= accuracy_score(predictions_train, Y_train)

In [34]:
print('accuracy_score train:',train_score )

accuracy_score train: 0.9874399038461539


In [36]:

predictions_test=model.predict(X_test)
test_score= accuracy_score(predictions_test, Y_test)

In [37]:
print('accuracy_score test:',test_score )

accuracy_score test: 0.9783653846153846


In [41]:
# predictive system

X_new=X_test[3] #buscamos la noticia que queremos analizar

prediction= model.predict(X_new)  #aplicamos la predicción
print (prediction)

if (prediction[0]==0):
    print('La noticia es REAL')  #si nos da 0 es real y sino es 0 (que será 1) es Falsa
else:
    print('La noticia es FALSA')

[0]
La noticia es REAL
